# Coursera Capstone project using 4 square API

In [127]:
import pandas as pd
import numpy as np
import requests
from lxml import html
import geocoder


In [128]:
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


-----------------------------------------------------------------------------
# Scraping Wikipedia for data PART 1 of SUBMISSION

Instead of making multiple scraping request its better to fetch whole table at once, find a pattern and clean the text.

In [129]:
#--------------------------Making request and fetching data form wiki
pageContent=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
tree = html.fromstring(pageContent.content)
table_data = tree.xpath('/html/body/div[3]/div[3]/div[4]/div/table[1]')
table_data = table_data[0].text_content()

--------------------------Pre-porcessing fetched data--------------------------

In [130]:
table_data=table_data.split('\n\n\n')
table_data.pop(0)

postal_code=[]
borough=[]
neig_hood=[]

for x in table_data:
    y=x.split('\n')
    if y[1]=='Not assigned':
        pass
    else:
        postal_code.append(y[0])
        borough.append(y[1])
        if y[2]=='Not assigned':
            neig_hood.append(y[1])
        else:
            neig_hood.append(y[2])

--------------------------Conversion to DataFrame--------------------------

In [ ]:
data_framee = pd.DataFrame(
    {'Postcode': postal_code,
     'Borough': borough,
     'Neighbourhood': neig_hood
    })
data_framee

#----------------------------------Merging both dataframes on postalcodes
Geospatial_Coordinates = pd.read_csv('/Users/aayushbhargava/Downloads/Geospatial_Coordinates.csv')
data_framee2 = pd.merge(left=data_framee,right=Geospatial_Coordinates, left_on='Postcode', right_on='Postal Code')
data_framee2=data_framee2.drop(columns='Postal Code')
print (data_framee2.shape)
data_framee2.head()

In [132]:
#--------------------------Cleaning Data frame
d = data_framee.groupby('Postcode', as_index=False).agg({'Borough':'unique','Neighbourhood':list})
d.sort_values(by=['Postcode'])
d['Neighbourhood']=d['Neighbourhood'].apply(','.join)
d['Borough']=d['Borough'].apply(''.join)
d

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


GeoCoder is not Working

'''
lat_lng_coords=None
while(lat_lng_coords is None):
    g = geocoder.google('M9M, Toronto, Ontario')
    lat_lng_coords = g.latlng
lat_lng_coords
print (lat_lng_coords[0])
print (lat_lng_coords[1])
'''

-----------------------------------------------------------------------------
# Adding lat,long. from csv PART 2 of SUBMISSION

In [135]:
#----------------------------------Merging both dataframes on postalcodes
merged_inner = pd.merge(left=d,right=Geospatial_Coordinates, left_on='Postcode', right_on='Postal Code')
merged_inner=merged_inner.drop(columns='Postal Code')
print (merged_inner.shape)
merged_inner.head()

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


-----------------------------------------------------------------------------
# Mapping Scarborough Data PART 3 of SUBMISSION

In [243]:
scarborough_data = data_framee2[data_framee2['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497


In [139]:
from geopy.geocoders import Nominatim
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [140]:
import folium
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Borough'], scarborough_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

---------------------------------
# Testing foursquare API
Using this API we can extract all data in JSON format and further clean it to extract venue catorory necessary for Analysis

In [148]:
CLIENT_ID = 'NRG5PBEEQPS0UJWCIWNWE3HABFW1GTHYTNLIZ3MU55PKJAZH' # your Foursquare ID
CLIENT_SECRET = 'U20WXVIWWKHEA5IS4S4LIWTPK3LZSWGW3HA3RDQ14TGQI2J3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


neighborhood_latitude = scarborough_data.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[3, 'Longitude'] # neighborhood longitude value
neighborhood_name = scarborough_data.loc[3, 'Neighbourhood'] # neighborhood name


def getNearbyVenues(names, latitudes, longitudes, radius=500):   
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )


Rouge
Malvern
Highland Creek
Rouge Hill
Port Union
Guildwood
Morningside
West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park
Ionview
Kennedy Park
Clairlea
Golden Mile
Oakridge
Cliffcrest
Cliffside
Scarborough Village West
Birch Cliff
Cliffside West
Dorset Park
Scarborough Town Centre
Wexford Heights
Maryvale
Wexford
Agincourt
Clarks Corners
Sullivan
Tam O'Shanter
Agincourt North
L'Amoreaux East
Milliken
Steeles East
L'Amoreaux West
Upper Rouge


In [149]:
scarborough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rouge,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Malvern,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Rouge Hill,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,Port Union,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar


### One Hot Encoding to get data ready for kmeans clustering

In [151]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Malvern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Highland Creek,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rouge Hill,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Port Union,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guildwood,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
6,Guildwood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guildwood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Guildwood,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,Guildwood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Finding mean of Data to get frequency of neighbourhood vs venue category 

In [152]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000000,0.000000,0.000,0.0,0.250000,0.0,0.00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.0,0.000,0.000,0.000
1,Agincourt North,0.0,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.00,...,0.000000,0.5,0.000000,0.000000,0.000000,0.00,0.0,0.000,0.000,0.000
2,Birch Cliff,0.0,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.0,0.000,0.000,0.000
3,Cedarbrae,0.0,0.125,0.000000,0.125000,0.125,0.0,0.000000,0.0,0.00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.125,0.000,0.000
4,Clairlea,0.0,0.000,0.000000,0.200000,0.000,0.0,0.000000,0.2,0.10,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.1,0.000,0.000,0.000
5,Clarks Corners,0.0,0.000,0.000000,0.000000,0.100,0.0,0.000000,0.0,0.00,...,0.200000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.100,0.000,0.000
6,Cliffcrest,0.5,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000,0.000,0.000
7,Cliffside,0.5,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000,0.000,0.000
8,Cliffside West,0.0,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.0,0.000,0.000,0.000
9,Dorset Park,0.0,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000,0.125,0.125


In [240]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted

# Clustering The data to groups PART 4 of Submission

In [241]:
from sklearn.cluster import KMeans
kclusters = 10

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([6, 3, 9, 8, 8, 6, 5, 5, 9, 1, 7, 8, 6, 2, 7, 7, 3, 6, 4, 8, 3, 6,
       8, 2, 4, 2, 1, 0, 5, 3, 6, 6, 6, 8, 1, 7], dtype=int32)

In [242]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
scarborough_merged=scarborough_merged.dropna()
scarborough_merged['Cluster Labels'] = scarborough_merged['Cluster Labels'].apply(np.int64)
#scarborough_merged   # check the last columns!

### Mapping the Cluster

In [212]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [229]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,0,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [230]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Scarborough,1,Indian Restaurant,Pet Store,Chinese Restaurant,Furniture / Home Store,Thrift / Vintage Store,Light Rail Station,Vietnamese Restaurant,Rental Car Location,Playground,Fried Chicken Joint
23,Scarborough,1,Indian Restaurant,Pet Store,Chinese Restaurant,Furniture / Home Store,Thrift / Vintage Store,Light Rail Station,Vietnamese Restaurant,Rental Car Location,Playground,Fried Chicken Joint
24,Scarborough,1,Indian Restaurant,Pet Store,Chinese Restaurant,Furniture / Home Store,Thrift / Vintage Store,Light Rail Station,Vietnamese Restaurant,Rental Car Location,Playground,Fried Chicken Joint


In [231]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,2,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
3,Scarborough,2,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
4,Scarborough,2,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store


In [232]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Scarborough,3,Playground,Park,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
32,Scarborough,3,Playground,Park,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
33,Scarborough,3,Playground,Park,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
34,Scarborough,3,Playground,Park,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store


In [233]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Department Store,Cosmetics Shop
1,Scarborough,4,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Department Store,Cosmetics Shop


In [234]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==5, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Scarborough,5,American Restaurant,Motel,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
18,Scarborough,5,American Restaurant,Motel,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
19,Scarborough,5,American Restaurant,Motel,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store


In [235]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==6, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,6,Mexican Restaurant,Intersection,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Vietnamese Restaurant,Furniture / Home Store,Fried Chicken Joint
6,Scarborough,6,Mexican Restaurant,Intersection,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Vietnamese Restaurant,Furniture / Home Store,Fried Chicken Joint
7,Scarborough,6,Mexican Restaurant,Intersection,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Vietnamese Restaurant,Furniture / Home Store,Fried Chicken Joint
27,Scarborough,6,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Vietnamese Restaurant,College Stadium,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
28,Scarborough,6,Pizza Place,Noodle House,Chinese Restaurant,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Bank,Fast Food Restaurant,Convenience Store,Furniture / Home Store
29,Scarborough,6,Pizza Place,Noodle House,Chinese Restaurant,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Bank,Fast Food Restaurant,Convenience Store,Furniture / Home Store
30,Scarborough,6,Pizza Place,Noodle House,Chinese Restaurant,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Bank,Fast Food Restaurant,Convenience Store,Furniture / Home Store
35,Scarborough,6,Chinese Restaurant,Fast Food Restaurant,Gym Pool,Breakfast Spot,Grocery Store,Noodle House,Cosmetics Shop,Pharmacy,Pizza Place,Electronics Store


In [236]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==7, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,7,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
11,Scarborough,7,Chinese Restaurant,Coffee Shop,Department Store,Bus Station,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
12,Scarborough,7,Chinese Restaurant,Coffee Shop,Department Store,Bus Station,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
13,Scarborough,7,Chinese Restaurant,Coffee Shop,Department Store,Bus Station,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [245]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==8, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,8,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Lounge,Convenience Store,Grocery Store
14,Scarborough,8,Bakery,Bus Line,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Park,Metro Station,Vietnamese Restaurant,College Stadium
15,Scarborough,8,Bakery,Bus Line,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Park,Metro Station,Vietnamese Restaurant,College Stadium
16,Scarborough,8,Bakery,Bus Line,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Park,Metro Station,Vietnamese Restaurant,College Stadium
25,Scarborough,8,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,College Stadium,General Entertainment,Furniture / Home Store
26,Scarborough,8,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,College Stadium,General Entertainment,Furniture / Home Store


In [246]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==9, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Scarborough,9,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
21,Scarborough,9,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant


### The cluster are perfetly divided as we can see 1st most common venue is mostly seperated in each cluster.
### Every cluster hs its own venue category and if we have to further divide it, it would have done even better